In [1]:
import pandas as pd
import os
from utility_functions import get_acc, add_row_to_csv

In [2]:
folder_path = 'training_data'
datasets = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
datasets.remove('detailed')
datasets.remove('systematic')

In [3]:
for dataset in datasets:
    cv_df = pd.read_csv(f"model/mlp/report_{dataset}.csv")
    evaluation_df = pd.read_csv('training_data/' + dataset + '/evaluation.csv')
    fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')

    for test_fold in sorted(fold_df['fold'].unique()):
        df_fold = cv_df[cv_df['test_fold'] == test_fold]
        
        eval_df = evaluation_df[evaluation_df['sequenceID'].isin(fold_df[fold_df['fold'] == test_fold]['sequenceID'])]
        chosen_row = df_fold.loc[df_fold['val_loss'].idxmin()]
        n_layer = chosen_row['num_layers']
        layer_size = chosen_row['layer_size']
        pred_df = pd.read_csv(f'model/mlp/predictions_all/{dataset}.{n_layer}layers_{layer_size}neurons_{test_fold}.csv')
        pred_df.fillna(0, inplace=True)
        acc = get_acc(eval_df, pred_df)
        add_row_to_csv('acc_rate_csvs/' + dataset + '.csv', ['method', 'fold', 'acc'], ['mlp', test_fold, acc])